In [36]:

import numpy as np
import altair as alt
import pandas as pd

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import streamlit as st

In [37]:
host = st.secrets['snowflake']['host']
user = st.secrets['snowflake']['user']
password = st.secrets['snowflake']['password']
account = st.secrets['snowflake']['account']
warehouse = st.secrets['snowflake']['warehouse']
database = st.secrets['snowflake']['database']
schema = st.secrets['snowflake']['schema']

cd = {
    'host' : host,
    'user' : user,
    'password' : password,
    'account' : account,
    'warehouse' : warehouse,
    'database' : database,
    'schema' : schema
}

In [38]:
def snowflake_query(snowflake_schema, snowflake_table, cd):

    query = f"select * from {snowflake_schema}.{snowflake_table}"

    connector = snowflake.connector.connect(
        host=cd['host'],
        user=cd['user'],
        password=cd['password'],
        account=cd['account'],
        warehouse=cd['warehouse'],
        database=cd['database'],
        schema=cd['schema'])

    try:
        cursor = connector.cursor().execute(query)
    finally:
        connector.close()
    df = cursor.fetch_pandas_all()

    return df

In [39]:
raw_places = snowflake_query('RAW', 'CSV__PLACES_DATA', cd)
raw_transit = snowflake_query('RAW', 'CSV__TRANSIT_DATA', cd)
raw_index_meta = snowflake_query('RAW', 'JSON__INDEX_META', cd)
raw_property_details = snowflake_query('RAW', 'JSON__DETAILS', cd)

2022-12-10 18:21:28.694 Snowflake Connector for Python Version: 2.7.12, Python Version: 3.9.13, Platform: macOS-10.16-x86_64-i386-64bit
2022-12-10 18:21:28.695 This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-10 18:21:30.154 query: [select * from RAW.CSV__PLACES_DATA]
2022-12-10 18:21:30.904 query execution done
2022-12-10 18:21:30.906 closed
2022-12-10 18:21:31.125 No async queries seem to be running, deleting session
2022-12-10 18:21:31.390 Failed to add log to telemetry.
Traceback (most recent call last):
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/telemetry.py", line 96, in try_add_log_to_batch
    self.add_log_to_batch(telemetry_data)
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/

In [40]:
stg_details = snowflake_query('STAGING', 'STG__DETAILS', cd)
stg_developers = snowflake_query('STAGING', 'STG__DEVELOPERS', cd)
stg_features = snowflake_query('STAGING', 'STG__FEATURES', cd)
stg_places = snowflake_query('STAGING', 'STG__PLACES', cd)
stg_projects = snowflake_query('STAGING', 'STG__PROJECTS_LIST', cd)
stg_transit = snowflake_query('STAGING', 'STG__TRANSIT', cd)
stg_vacancy = snowflake_query('STAGING', 'STG__VACANCY', cd)

2022-12-10 18:21:41.064 Snowflake Connector for Python Version: 2.7.12, Python Version: 3.9.13, Platform: macOS-10.16-x86_64-i386-64bit
2022-12-10 18:21:41.065 This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-10 18:21:42.597 query: [select * from STAGING.STG__DETAILS]
2022-12-10 18:21:43.014 query execution done
2022-12-10 18:21:43.015 closed
2022-12-10 18:21:43.216 No async queries seem to be running, deleting session
2022-12-10 18:21:43.473 Failed to add log to telemetry.
Traceback (most recent call last):
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/telemetry.py", line 96, in try_add_log_to_batch
    self.add_log_to_batch(telemetry_data)
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/

In [41]:
raw_places.columns

Index(['_AIRBYTE_UNIQUE_KEY', 'NAME', 'FSQ_ID', '_AB_SOURCE_FILE_URL',
       'geocodes.main.latitude', 'geocodes.main.longitude',
       '_AB_ADDITIONAL_PROPERTIES', 'location.formatted_address',
       '_AB_SOURCE_FILE_LAST_MODIFIED', '_AIRBYTE_AB_ID',
       '_AIRBYTE_EMITTED_AT', '_AIRBYTE_NORMALIZED_AT',
       '_AIRBYTE_CSV__PLACES_DATA_HASHID'],
      dtype='object')

In [42]:
raw_places[['NAME', 'geocodes.main.latitude', 'geocodes.main.longitude', 'location.formatted_address','FSQ_ID', 
            '_AIRBYTE_UNIQUE_KEY', '_AB_SOURCE_FILE_LAST_MODIFIED', '_AIRBYTE_AB_ID','_AIRBYTE_EMITTED_AT', 
            '_AIRBYTE_NORMALIZED_AT', '_AIRBYTE_CSV__PLACES_DATA_HASHID']]

,NAME,geocodes.main.latitude,geocodes.main.longitude,location.formatted_address,FSQ_ID,_AIRBYTE_UNIQUE_KEY,_AB_SOURCE_FILE_LAST_MODIFIED,_AIRBYTE_AB_ID,_AIRBYTE_EMITTED_AT,_AIRBYTE_NORMALIZED_AT,_AIRBYTE_CSV__PLACES_DATA_HASHID
0,MaxValu (แม็กซ์แวลู),13.639875,100.501008,"ทุ่งครุ, Bangkok 10140",4c980321d799a1cd6f02af52,010a891964ec638f143ce9418cce59e8,2022-12-01 05:06:15-08:00,ba94407b-79d6-4af4-ab48-297939791bf6,2022-12-01 13:21:08.107000-08:00,2022-12-01 05:22:35.374000-08:00,c16eb3fba7414f0b2eaa3a4c7b931ef6
1,Makro (แม็คโคร),13.649935,100.642390,"สมุทรปราการ, Samut Prakan 10270",4b6f9cb5f964a520e1f72ce3,02a2a7fcb327dbccd8c1050301555fd1,2022-12-01 05:06:13-08:00,ddd9a214-dd9f-4785-b8ed-a7cd8abc97d4,2022-12-01 13:21:01.993000-08:00,2022-12-01 05:22:35.374000-08:00,35da2aefbdbc7c1238e401ca9d8967e1
2,Jodd Fairs (จ๊อดแฟร์),13.756861,100.566704,"ห้วยขวาง, Bangkok 10310",61a7008421d37b3e23586d59,035a976f9203d8f71b134a2057d6d884,2022-12-01 05:06:18-08:00,66963ac1-5abe-44d5-a1b1-c81e023d68b6,2022-12-01 13:21:16.055000-08:00,2022-12-01 05:22:35.374000-08:00,4b70ff37f126f0b5629a77ca294bf531
3,Haidilao (ไหตี่เลา),13.757937,100.566390,"7th Fl, #704, ห้วยขวาง, Bangkok",5f1d70337f88c415c2941410,054b21492100a3037ef1c3244a20ac7e,2022-12-01 05:06:18-08:00,714fa5b7-7291-46a1-a4f3-3b0c11ee039a,2022-12-01 13:21:16.056000-08:00,2022-12-01 05:22:35.374000-08:00,7856fac0f1e582372ef929882d2d7c19
4,Att U Park (แอท ยู พาร์ค),13.632308,100.716197,"km 12, Samut Prakan 10540",5b6e5507531593003908d74c,09dddba7806760b9650d0437c26e9924,2022-12-01 05:06:13-08:00,3330deab-273d-4db1-aedc-84210e220289,2022-12-01 13:21:02-08:00,2022-12-01 05:22:35.374000-08:00,13059984d5684079352c76803a00e62c
...,...,...,...,...,...,...,...,...,...,...,...
91,Tesco Lotus (เทสโก้ โลตัส),13.818085,100.563197,"จตุจักร, Bangkok 10900",4c009ff9360a9c74d801d9a0,f757adc77e6bfebf7163af3c97c234a6,2022-12-01 05:06:18-08:00,ff3f66ea-f469-4de3-88ab-bc9bd060e4f6,2022-12-01 13:21:16.064000-08:00,2022-12-01 05:22:35.374000-08:00,6caa20d606e4a0e935665a050148d85a
92,Big C (บิ๊กซี),13.826917,100.680024,"คันนายาว, Bangkok 10230",4ccc19dbc0378cfa1e9d8348,f77d930f95229eb8890f7c922499840d,2022-12-01 05:06:16-08:00,891084fa-725b-44ee-8f66-d49ff0f16f81,2022-12-01 13:21:11.942000-08:00,2022-12-01 05:22:35.374000-08:00,22d2b2d0f012f212b335d220f4558908
93,Donki Mall (ดองกิ มอลล์),13.730795,100.585840,"Soi Sukhumvit 63, วัฒนา, Bangkok 10110",5c6cc4013ba767002b98f8cc,fa2694f4fc0136639c4cf0d8ae50978b,2022-12-01 05:06:18-08:00,d5ac5890-8713-4fb4-92e6-bab8ddbd78c3,2022-12-01 13:21:16.051000-08:00,2022-12-01 05:22:35.374000-08:00,07afd0a7772d70a6a0ed56e9d123ede1
94,CQK Hot Pot (CQK Hotpot ห้วยขวาง),13.767790,100.574141,"ห้วยขวาง, Bangkok",618cedfbb87f97720bbb37a6,fe5953a6111ac560002d43893c4a2fb6,2022-12-01 05:06:18-08:00,d2b49265-4157-420a-a91b-5337b4f2e14b,2022-12-01 13:21:16.058000-08:00,2022-12-01 05:22:35.374000-08:00,40861d880594d5e5697a7fa8a7bd2ecc


In [43]:
raw_transit.columns

Index(['LAT', 'CODE', 'LONG', 'NAME', '_AB_SOURCE_FILE_URL',
       '_AB_ADDITIONAL_PROPERTIES', '_AB_SOURCE_FILE_LAST_MODIFIED',
       '_AIRBYTE_AB_ID', '_AIRBYTE_EMITTED_AT', '_AIRBYTE_NORMALIZED_AT',
       '_AIRBYTE_CSV__TRANSIT_DATA_HASHID'],
      dtype='object')

In [44]:
raw_index_meta

,_AIRBYTE_UNIQUE_KEY,PROJECT_URL,PROJECT_NAME,PROJECT_RENT,PROJECT_SALE,_AB_SOURCE_FILE_URL,_AB_ADDITIONAL_PROPERTIES,_AB_SOURCE_FILE_LAST_MODIFIED,_AIRBYTE_AB_ID,_AIRBYTE_EMITTED_AT,_AIRBYTE_NORMALIZED_AT,_AIRBYTE_JSON__INDEX_META_HASHID
0,0073d790e914c5095638463b55ba9f37,https://www.thailand-property.com/condo/1340/c...,CitiSmart Sukhumvit 18,213,37,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:25-08:00,32b6d410-05de-4233-87cc-ddccfa87c054,2022-12-04 15:25:41.485000-08:00,2022-12-04 07:27:01.315000-08:00,df71a3beeede46f9b49fb53ca8cf9b64
1,008d0df54674c9e729b95de1cfcab7ea,https://www.thailand-property.com/condo/13419/...,U&I Mansion,2,0,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:12-08:00,f11db61c-8a1b-4bce-9200-b3dcb0642524,2022-12-04 15:25:37.590000-08:00,2022-12-04 07:27:01.315000-08:00,4af498dc29d4194c91aea03c32d29224
2,00a843ced225740371be7d2ece14fe2a,https://www.thailand-property.com/condo/15076/...,The Tree Sukhumvit - Rama 4,0,6,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:25-08:00,b00238a7-1534-40c1-a236-c2a05c5846da,2022-12-04 15:25:41.443000-08:00,2022-12-04 07:27:01.315000-08:00,1e964046b5de7f35555a8669427eec6f
3,01076bd223fce838f673562eb58ebe51,https://www.thailand-property.com/condo/11995/...,The Teak Sukhumvit 39,47,10,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:06-08:00,e91f035d-443c-42d6-b497-4c956b6d5997,2022-12-04 15:25:33.553000-08:00,2022-12-04 07:27:01.315000-08:00,34e2c197dfd689bfe3132849580ee663
4,010c48bccacbe6091375a9b0ec3bc606,https://www.thailand-property.com/condo/1413/d...,Domus,103,12,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:25-08:00,d2b91cd6-975e-4ce2-8ed3-7fa69738e5bb,2022-12-04 15:25:41.487000-08:00,2022-12-04 07:27:01.315000-08:00,a4ac8e25ec482f6f41c08be5b8e27531
...,...,...,...,...,...,...,...,...,...,...,...,...
995,fee65f2164a0a37adc76a143a729caec,https://www.thailand-property.com/condo/14154/...,Aspire Asoke-Ratchada,18,17,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:08-08:00,ffea030f-698c-4ffe-aee9-10186a4e9dbf,2022-12-04 15:25:35.516000-08:00,2022-12-04 07:27:01.315000-08:00,a1c850808aac96540ac9f508c82d1f58
996,fee9b27d6e1b0efc6b014e700b4a0411,https://www.thailand-property.com/condo/18918/...,COCO Parc,0,24,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:12-08:00,724d4510-5090-447b-ac03-f98bb66c255d,2022-12-04 15:25:37.572000-08:00,2022-12-04 07:27:01.315000-08:00,ec642fa918fe078808a8d422ba753fca
997,ff003482e95d86b4dc390f5d81d62a32,https://www.thailand-property.com/condo/2011/r...,Rhythm Sathorn,174,118,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:16-08:00,7dbef6e0-5328-4994-b49c-3def6f588dec,2022-12-04 15:25:39.559000-08:00,2022-12-04 07:27:01.315000-08:00,25372837e2cdd84c83eaf7e535b72a1c
998,ff14d3122c3e27bf7d4c922ca8b931d5,https://www.thailand-property.com/condo/12401/...,Knightsbridge Kaset Society,5,3,topics/index_meta/year=2022/month=12/day=04/ho...,"{\n ""_ab_source_file_last_modified"": null,\n ...",2022-12-04 07:00:16-08:00,75b4f3bb-eb5d-45db-88cb-5566a25ff53f,2022-12-04 15:25:39.546000-08:00,2022-12-04 07:27:01.315000-08:00,22d529061433f7f4c3bae32a78bf7e30


In [45]:
raw_index_meta.columns

Index(['_AIRBYTE_UNIQUE_KEY', 'PROJECT_URL', 'PROJECT_NAME', 'PROJECT_RENT',
       'PROJECT_SALE', '_AB_SOURCE_FILE_URL', '_AB_ADDITIONAL_PROPERTIES',
       '_AB_SOURCE_FILE_LAST_MODIFIED', '_AIRBYTE_AB_ID',
       '_AIRBYTE_EMITTED_AT', '_AIRBYTE_NORMALIZED_AT',
       '_AIRBYTE_JSON__INDEX_META_HASHID'],
      dtype='object')

In [46]:
raw_property_details.columns

Index(['PROJECT_MAP', 'PROJECT_URL', 'PROJECT_NAME', 'PROJECT_RENT',
       'PROJECT_SALE', 'PROJECT_YEAR', 'PROJECT_FLOORS', 'PROJECT_FEATURES',
       'PROJECT_DEVELOPER', '_AB_SOURCE_FILE_URL', '_AB_ADDITIONAL_PROPERTIES',
       '_AB_SOURCE_FILE_LAST_MODIFIED', '_AIRBYTE_AB_ID',
       '_AIRBYTE_EMITTED_AT', '_AIRBYTE_NORMALIZED_AT',
       '_AIRBYTE_JSON__DETAILS_HASHID', '_AIRBYTE_UNIQUE_KEY'],
      dtype='object')

In [53]:
stg_projects

,PROJECT_KEY,NAME,URL
0,0073d790e914c5095638463b55ba9f37,CitiSmart Sukhumvit 18,https://www.thailand-property.com/condo/1340/c...
1,008d0df54674c9e729b95de1cfcab7ea,U&I Mansion,https://www.thailand-property.com/condo/13419/...
2,00a843ced225740371be7d2ece14fe2a,The Tree Sukhumvit - Rama 4,https://www.thailand-property.com/condo/15076/...
3,01076bd223fce838f673562eb58ebe51,The Teak Sukhumvit 39,https://www.thailand-property.com/condo/11995/...
4,010c48bccacbe6091375a9b0ec3bc606,Domus,https://www.thailand-property.com/condo/1413/d...
...,...,...,...
995,fee65f2164a0a37adc76a143a729caec,Aspire Asoke-Ratchada,https://www.thailand-property.com/condo/14154/...
996,fee9b27d6e1b0efc6b014e700b4a0411,COCO Parc,https://www.thailand-property.com/condo/18918/...
997,ff003482e95d86b4dc390f5d81d62a32,Rhythm Sathorn,https://www.thailand-property.com/condo/2011/r...
998,ff14d3122c3e27bf7d4c922ca8b931d5,Knightsbridge Kaset Society,https://www.thailand-property.com/condo/12401/...


In [52]:
raw_property_details['_AIRBYTE_EMITTED_AT'].max().strftime('%d-%m-%Y %H:%M:%S')

'06-12-2022 05:15:57'

In [59]:
stg_transit.columns

Index(['TRANSIT_KEY', 'CODE', 'NAME', 'LAT', 'LONG', 'COORD'], dtype='object')